Pipeline for Extracting Keywords from the dataset

In [210]:
import pandas as pd
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
#import nltk
#nltk.download('punkt')


In [211]:
nlp = spacy.load("en_core_web_sm")

path = "/Users/ethanvirtudazo/Desktop/DS105_Dataset/6622_jobs.xls"
df_import = pd.read_excel(path)

df_import.shape

(6623, 11)

In [214]:
df_import.head(2)

,Unnamed: 0,title,details,deadline,opport_type,commence_date,contract_type,location,Renumeration,company,links
0,0,Rothschild & Co - Private Equity Long-Term Int...,This London-based 6-month internship is an exc...,2023-04-30,Internship,2023-07-01 00:00:00,Temporary,London,NaN,Rothschild & Co,https://careers.lse.ac.uk//students/jobs/detai...
1,1,2023 HSBC Global Graduate Programme (Hong Kong...,You’re excited about starting your career and ...,2023-01-06,Graduate employment,2023-07-03 00:00:00,Temporary,Hong KongSingapore,NaN,HSBC (HSBC) - Hong Kong,https://careers.lse.ac.uk//students/jobs/detai...


In [215]:
df1 = df_import.iloc[:, 1:5]
df1.head()

df2 = df_import.iloc[:, [7,9,10]]
df2.head()

pdList = [df1,df2]
df = pd.concat(pdList,axis=1)
df.head()

df.shape

types = df.dtypes
print(types)

df_title = df.iloc[:,0]
df_text_both = df.iloc[:, [0,1]]
df_text_1 = df.iloc[:, [1]]

df_text_both.head()


title                  object
details                object
deadline       datetime64[ns]
opport_type            object
location               object
company                object
links                  object
dtype: object


,title,details
0,Rothschild & Co - Private Equity Long-Term Int...,This London-based 6-month internship is an exc...
1,2023 HSBC Global Graduate Programme (Hong Kong...,You’re excited about starting your career and ...
2,2023 HSBC Global Internship Programme (Hong Ko...,You’re excited about starting your career and ...
3,"Graduate Training Scheme, Capital Markets",Graduate Training Scheme – LondonGreySpark Par...
4,6-Months Internship – Sell-side Tech M&A,"At IPTP, we understand software from decades o..."


In [216]:
df_text_1.head()

,details
0,This London-based 6-month internship is an exc...
1,You’re excited about starting your career and ...
2,You’re excited about starting your career and ...
3,Graduate Training Scheme – LondonGreySpark Par...
4,"At IPTP, we understand software from decades o..."


In [217]:
len(df_text_1)

6623

In [218]:
type(df_text_1)

pandas.core.frame.DataFrame

In [219]:
#ar_text = df_text.to_numpy()
#ar_text = ar_text[:101]

#Subset of 100 jobs
df_text = df_text_1[:101]

In [220]:
#All Jobs
df_text_all = df_text_1

In [221]:
len(df_text_all)

6623

In [13]:
#Subset of data
txt_lst = df_text['details'].tolist()

#docs = list(nlp.pipe(df_text['details']))
docs = df_text['details'].apply(nlp)

#docs = [nlp(string) for string in df_text]
#docs = list(nlp.pipe(df_text))

In [14]:
len(docs)

101

In [208]:
len(df_text_all)

6623

In [222]:
#converting ALL 6600+ jobs into spaCy document
all_txt_lst = df_text_all['details'].tolist()
len(all_txt_lst)
#ls = [type(item) for item in all_txt_lst]

6623

In [223]:
# Locating jobs without text (data value not class "str")
indices = [i for i, x in enumerate(all_txt_lst) if not isinstance(x, str)]
print(indices)

[1800, 3395, 6081]


In [226]:
all_txt_df = pd.DataFrame(all_txt_lst)

In [228]:
#Removing rows with values that are not text (class 'str')
all_txt_df = all_txt_df.drop([all_txt_df.index[1800], all_txt_df.index[3395], all_txt_df.index[6081]])

In [230]:
len(all_txt_df)

6620

In [233]:
all_txt_df.columns =['details']

In [234]:
all_txt_df.head()

,details
0,This London-based 6-month internship is an exc...
1,You’re excited about starting your career and ...
2,You’re excited about starting your career and ...
3,Graduate Training Scheme – LondonGreySpark Par...
4,"At IPTP, we understand software from decades o..."


In [235]:
all_docs = all_txt_df['details'].apply(nlp)

In [236]:
len(all_docs)


6620

In [240]:
all_docs_1 = all_docs[0:1000]
all_docs_2 = all_docs[1000:2000]
all_docs_3 = all_docs[2000:3000]
all_docs_4 = all_docs[3000:4000]
all_docs_5 = all_docs[4000:5000]
all_docs_6 = all_docs[5000:6000]
all_docs_7 = all_docs[6000:6620]

In [254]:
#Individual Job View
rend_doc = all_docs_7[6615]

from spacy import displacy
displacy.render(rend_doc,style="ent")

In [255]:
# This code outputs the sentences for a particular document.
for sent in all_docs_7[6615].sents:
    print("sentence:\n",sent)

sentence:
 Business: Financial Systems Area: Capital Markets - Adaptiv Location: Cambridge/Oxford UK Level:
sentence:
 Undergraduate, PostgraduateSunGard's Adaptiv provides enterprise-wide credit and market risk management and operations solutions for financial services institutions.
sentence:
 Adaptiv assists institutions of varying size and complexity to deploy technology to meet both internal and regulatory requirements for risk management and operational control.
sentence:
 Adaptiv helps financial services institutions from the banking, hedge fund, asset management, insurance and corporate sectors with our deep understanding of risk management and operational processes.
sentence:
 The problems Adaptiv solves are inherently quantitative and rich in technical innovation.
sentence:
 We combine sophisticated mathematical techniques with leading edge technologies, to address today's demanding regulatory and reporting challenges in the financial environment.
sentence:
 Find out more at w

In [39]:
first_doc_sent = [sent.text for sent in docs[0].sents]
len(first_doc_sent)

7

In [145]:
#all_sent contains all of the sentences
all_sent_sub = [[sent.text for sent in doc.sents] for doc in docs]
len(all_sent_sub)

101

In [150]:
#viewing one particular sentence and its tokens and the tokens respective pos
sent_tokens = []

for token in docs[0]:
    tok_pair = [token, token.pos_]
    sent_tokens.append(tok_pair)


In [151]:
sent_tokens

[[This, 'DET'],
 [London, 'PROPN'],
 [-, 'PUNCT'],
 [based, 'VERB'],
 [6, 'NUM'],
 [-, 'PUNCT'],
 [month, 'NOUN'],
 [internship, 'NOUN'],
 [is, 'AUX'],
 [an, 'DET'],
 [exciting, 'ADJ'],
 [opportunity, 'NOUN'],
 [to, 'PART'],
 [intern, 'VERB'],
 [with, 'ADP'],
 [Rothschild, 'PROPN'],
 [&, 'CCONJ'],
 [Co, 'PROPN'],
 ['s, 'PART'],
 [European, 'ADJ'],
 [corporate, 'ADJ'],
 [private, 'ADJ'],
 [equity, 'NOUN'],
 [business, 'NOUN'],
 [., 'PUNCT'],
 [Rothschild, 'PROPN'],
 [&, 'CCONJ'],
 [Co, 'PROPN'],
 [’s, 'PART'],
 [European, 'ADJ'],
 [corporate, 'ADJ'],
 [private, 'ADJ'],
 [equity, 'NOUN'],
 [business, 'NOUN'],
 [is, 'AUX'],
 [comprised, 'VERB'],
 [of, 'ADP'],
 [three, 'NUM'],
 [strategies, 'NOUN'],
 [:, 'PUNCT'],
 [Five, 'NUM'],
 [Arrows, 'NOUN'],
 [Principal, 'PROPN'],
 [Investments, 'PROPN'],
 [(, 'PUNCT'],
 [FAPI, 'PROPN'],
 [), 'PUNCT'],
 [,, 'PUNCT'],
 [Five, 'NUM'],
 [Arrows, 'PROPN'],
 [Growth, 'PROPN'],
 [Capital, 'PROPN'],
 [(, 'PUNCT'],
 [FAGC, 'PROPN'],
 [), 'PUNCT'],
 [,, 'PUN

In [153]:
sent_tokens = []

for token in docs[5]:
    tok_pair = [token, token.pos_]
    sent_tokens.append(tok_pair)

sent_tokens

[[Our, 'PRON'],
 [Global, 'PROPN'],
 [Markets, 'PROPN'],
 [business, 'NOUN'],
 [offers, 'VERB'],
 [sales, 'NOUN'],
 [and, 'CCONJ'],
 [trading, 'NOUN'],
 [services, 'NOUN'],
 [,, 'PUNCT'],
 [including, 'VERB'],
 [research, 'NOUN'],
 [,, 'PUNCT'],
 [to, 'ADP'],
 [institutional, 'ADJ'],
 [clients, 'NOUN'],
 [across, 'ADP'],
 [fixed, 'VERB'],
 [-, 'PUNCT'],
 [income, 'NOUN'],
 [,, 'PUNCT'],
 [credit, 'NOUN'],
 [,, 'PUNCT'],
 [currency, 'NOUN'],
 [,, 'PUNCT'],
 [and, 'CCONJ'],
 [commodity, 'NOUN'],
 [and, 'CCONJ'],
 [equity, 'NOUN'],
 [businesses, 'NOUN'],
 [., 'PUNCT'],
 [Global, 'PROPN'],
 [Markets, 'PROPN'],
 [product, 'NOUN'],
 [coverage, 'NOUN'],
 [includes, 'VERB'],
 [securities, 'NOUN'],
 [and, 'CCONJ'],
 [derivative, 'ADJ'],
 [products, 'NOUN'],
 [in, 'ADP'],
 [both, 'CCONJ'],
 [the, 'DET'],
 [primary, 'ADJ'],
 [and, 'CCONJ'],
 [secondary, 'ADJ'],
 [markets, 'NOUN'],
 [., 'PUNCT'],
 [The, 'DET'],
 [Global, 'PROPN'],
 [Sustainable, 'PROPN'],
 [Finance, 'PROPN'],
 [Group, 'PROPN'],
 [

In [102]:
docs[0]

This London-based 6-month internship is an exciting opportunity to intern with Rothschild & Co's European corporate private equity business. Rothschild & Co’s European corporate private equity business is comprised of three strategies: Five Arrows Principal Investments (FAPI), Five Arrows Growth Capital (FAGC), and Five Arrows Long Term Fund (FALT). FAPI manages c. €1.3 billion through its latest fund and is the flagship corporate private equity / buy-out vehicle investing in mid-cap companies, while FAGC manages c. €500 million through growth investments and small-cap buy-outs, and FALT makes investments in larger companies with the option for longer-term holding periods. All three funds are focused on investments within Western Europe (with FALT additionally on North America) and primarily invest in three main verticals: Data & Software, Healthcare, and Technology-enabled Business Services. The respective teams are based in London, Paris and Luxembourg and are comprised of c. 30 inve

In [103]:
# PARTS OF SPEECH FILTER

#filtered_docs = list([[token.text for token in sent if token.pos_ in ['PROPN', 'NOUN','ADJ']] for doc in docs])

#filtered_docs = list([token.text for doc in docs for sent in doc.sents for token in sent if token.pos_ in ["PROPN", "NOUN", "ADJ"]])

filtered_docs = [[token.text for sent in doc.sents for token in sent if token.pos_ in ["PROPN", "NOUN", "ADJ"]] for doc in docs]


In [128]:
filtered_docs[0:3]

[['London',
  'month',
  'internship',
  'exciting',
  'opportunity',
  'Rothschild',
  'Co',
  'European',
  'corporate',
  'private',
  'equity',
  'business',
  'Rothschild',
  'Co',
  'European',
  'corporate',
  'private',
  'equity',
  'business',
  'strategies',
  'Arrows',
  'Principal',
  'Investments',
  'FAPI',
  'Arrows',
  'Growth',
  'Capital',
  'FAGC',
  'Arrows',
  'Long',
  'Term',
  'Fund',
  'FALT',
  'FAPI',
  'c.',
  'latest',
  'fund',
  'flagship',
  'corporate',
  'private',
  'equity',
  'vehicle',
  'mid',
  '-',
  'cap',
  'companies',
  'FAGC',
  'c.',
  'growth',
  'investments',
  'small',
  'cap',
  'buy',
  'outs',
  'FALT',
  'investments',
  'larger',
  'companies',
  'option',
  'longer',
  'term',
  'holding',
  'periods',
  'funds',
  'investments',
  'Western',
  'Europe',
  'FALT',
  'North',
  'America',
  'main',
  'verticals',
  'Data',
  'Software',
  'Healthcare',
  'Technology',
  'Business',
  'Services',
  'respective',
  'teams',
  'Lond

In [129]:
type(filtered_docs)

list

In [105]:
# Creating Data for Wordcloud
len(filtered_docs)  

101

In [135]:
filtered_docs[0]

['London',
 'month',
 'internship',
 'exciting',
 'opportunity',
 'Rothschild',
 'Co',
 'European',
 'corporate',
 'private',
 'equity',
 'business',
 'Rothschild',
 'Co',
 'European',
 'corporate',
 'private',
 'equity',
 'business',
 'strategies',
 'Arrows',
 'Principal',
 'Investments',
 'FAPI',
 'Arrows',
 'Growth',
 'Capital',
 'FAGC',
 'Arrows',
 'Long',
 'Term',
 'Fund',
 'FALT',
 'FAPI',
 'c.',
 'latest',
 'fund',
 'flagship',
 'corporate',
 'private',
 'equity',
 'vehicle',
 'mid',
 '-',
 'cap',
 'companies',
 'FAGC',
 'c.',
 'growth',
 'investments',
 'small',
 'cap',
 'buy',
 'outs',
 'FALT',
 'investments',
 'larger',
 'companies',
 'option',
 'longer',
 'term',
 'holding',
 'periods',
 'funds',
 'investments',
 'Western',
 'Europe',
 'FALT',
 'North',
 'America',
 'main',
 'verticals',
 'Data',
 'Software',
 'Healthcare',
 'Technology',
 'Business',
 'Services',
 'respective',
 'teams',
 'London',
 'Paris',
 'Luxembourg',
 'c.',
 'investment',
 'professionals',
 'countries

In [136]:
# Joining individual tokens back into one filtered sentence list for each job
filtered_sent = [" ".join(sent) for sent in filtered_docs]

In [137]:
filtered_sent[0]

'London month internship exciting opportunity Rothschild Co European corporate private equity business Rothschild Co European corporate private equity business strategies Arrows Principal Investments FAPI Arrows Growth Capital FAGC Arrows Long Term Fund FALT FAPI c. latest fund flagship corporate private equity vehicle mid - cap companies FAGC c. growth investments small cap buy outs FALT investments larger companies option longer term holding periods funds investments Western Europe FALT North America main verticals Data Software Healthcare Technology Business Services respective teams London Paris Luxembourg c. investment professionals countries Responsibilities industries potential targets investments sources information Identifying investment opportunities financial models investment committee memos standard financial models deal execution processes monitoring portfolio companies point view value actionable opportunities Fund risk return criteria team co - due diligence transaction

In [156]:
from spacy import displacy
displacy.render(nlp(filtered_sent[5]),style="ent")

In [138]:
filtered_sent_df = pd.DataFrame(filtered_sent)
type(filtered_sent_df)

pandas.core.frame.DataFrame

In [139]:
len(filtered_sent_df)

101

In [140]:
filtered_sent_df.head()

,0
0,London month internship exciting opportunity R...
1,excited career many paths possibilities global...
2,excited career many paths possibilities global...
3,Graduate Training Scheme LondonGreySpark Partn...
4,IPTP software decades deep experience technolo...


In [107]:
#subsetting titles dataframe
df_title_sub = df_title[0:101]
len(df_title_sub)

101

In [141]:
df_wcloud_test = pd.concat([df_title_sub, filtered_sent_df], axis = 1)
type(df_wcloud_test)
df_wcloud_test.shape

(101, 2)

In [143]:
df_wcloud_test.to_csv('df_wcloud_test.csv')

COMPLETE DATASET